In [8]:
import json
import os
from glob import glob
import pandas as pd
import re
import plotly.graph_objs as go

In [67]:
all_files = glob("./stats/*.json")
files = []
for f in all_files:
    temp = os.path.split(f)[-1]
    if re.search(r'cifar\_NIID', temp) is not None:
        files.append(f)
# print(files)
method_acc = []
method_total_flops = []
method_client_flops = []
method_comm = []
steps = []
names = []

# hparams = {}
# for k in json.load(open(files[0], "r"))['hparams'].keys():
#     hparams[k] = []
    

for f in files:
    j = json.load(open(f, "r"))
    method_acc.append(j['Method Accuracy'])
    method_total_flops.append(j['Method Flops'])
    method_client_flops.append(j['Method Client Flops'])
    method_comm.append(j['Method Comm Cost'])
    steps.append(j['Steps'])
    names.append(os.path.split(f)[-1].replace(".json", ""))
    
    # for k, v in j['hparams'].items():
    #     hparams[k].append(v)

df = pd.DataFrame(
    zip(names, method_acc, method_comm, method_total_flops, method_client_flops), 
    columns=["Experiment Name", "Method Accuracy", "Method Comm Cost", "Total Flops", "Client Flops"]
)
# for k, v in hparams.items():
#     df[k] = v + [None] * (len(files) - len(v))
df

,Experiment Name,Method Accuracy,Method Comm Cost,Total Flops,Client Flops
0,cifar_NIID-Vanilla-10-1010,"[0.32199999999999995, 0.24300000000000005, 0.2...","[4208.72191999985, 8417.443840000984, 12626.16...","[3862789898240, 7725579796480, 11588369694720,...","[1405304709120, 2810609418240, 4215914127360, ..."
1,cifar_NIID-CESL-3-1010,"[0.09783333333333333, 0.09783333333333333, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1405491609600, 2810983219200, 4216474828800, ...","[1405491609600, 2810983219200, 4216474828800, ..."
2,cifar_NIID-CESL-Random-6-1010,"[0.10200000000000001, 0.10200000000000001, 0.1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1405491609600, 2810983219200, 4216474828800, ...","[1405491609600, 2810983219200, 4216474828800, ..."
3,cifar_NIID-CESL-Random-3-1010,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1405491609600, 2810983219200, 4216474828800, ...","[1405491609600, 2810983219200, 4216474828800, ..."
4,cifar_NIID-CESL-6-1010,"[0.10283333333333333, 0.10283333333333333, 0.1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1405491609600, 2810983219200, 4216474828800, ...","[1405491609600, 2810983219200, 4216474828800, ..."


In [68]:
temp = pd.DataFrame()
# print(df.columns)
temp['Experiment Name'] = df['Experiment Name']
temp['Method Accuracy'] = df['Method Accuracy'].apply(lambda x: max(eval(x)))
temp['Client TFLOPS'] = df['Client Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
temp['Total TFLOPS'] = df['Total Flops'].apply(lambda x: eval(x)[-1] * 1e-12)
temp['Comm Cost (GBs)'] = df['Method Comm Cost'].apply(lambda x: eval(x)[-1] * 1e-6)
temp

,Experiment Name,Method Accuracy,Client TFLOPS,Total TFLOPS,Comm Cost (GBs)
0,cifar_NIID-Vanilla-10-1010,0.417500,210.795706,579.418485,0.631308
1,cifar_NIID-CESL-3-1010,0.887667,140.549161,170.038983,0.050505
2,cifar_NIID-CESL-Random-6-1010,0.830333,140.549161,199.528805,0.101009
3,cifar_NIID-CESL-Random-3-1010,0.839667,140.549161,170.038983,0.050505
4,cifar_NIID-CESL-6-1010,0.919667,140.549161,199.528805,0.101009


In [40]:
fig = go.Figure()

for i in range(len(df)):
    fig.add_trace(
        go.Scatter(x=eval(steps[i]), y=eval(df['Method Accuracy'][i]), name=df['Experiment Name'][i])
    )
fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

fig.update_layout(template=None, xaxis_title="Training Steps", yaxis_title="Per-Client Test Accuracy", font=dict(family='Times New Roman', size=14, color='Black'))
fig.show()

In [4]:
fig = go.Figure()

for i in range(len(df)):
    fig.add_trace(
        go.Scatter(x=eval(steps[i]), y=eval(df['Method Comm Cost'][i]), name=df['Experiment Name'][i])
    )
fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

fig.update_layout(template=None, xaxis_title="Training Steps", yaxis_title="Communication Cost", font=dict(family='Times New Roman', size=14, color='Black'))

fig.show()

In [5]:
fig = go.Figure()

for i in range(len(df)):
    fig.add_trace(
        go.Scatter(x=eval(steps[i]), y=eval(df['Method Flops'][i]), name=df['Experiment Name'][i])
    )
fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

fig.update_layout(template=None, xaxis_title="Training Steps", yaxis_title="Total Flops", font=dict(family='Times New Roman', size=14, color='Black'))

fig.show()

In [6]:
files = glob("./stats/comparison/*.json")
method_acc = []
method_flops = []
steps = []
names = []
method_comm = []

hparams = {}
for k in json.load(open(files[0], "r"))['hparams'].keys():
    hparams[k] = []

# def get_name(d):
#     cols = {
#         "use_ucb": "UCB",
#         "poll_clients": "Polling",
#         "use_contrastive": "CL",
#         "interrupted": "Interrupted"
#     }
#     wanted = []
#     for c, v in cols.items():
#         if d[c]:
#             wanted.append(v)
#     return "+".join(wanted)

for f in files:
    j = json.load(open(f, "r"))
    method_acc.append(j['Method Accuracy'])
    method_flops.append(j['Method Flops'])
    method_comm.append(j['Method Comm Cost'])
    steps.append(j['Steps'])
    names.append(os.path.split(f)[-1].replace(".json", ""))

    for k, v in j['hparams'].items():
        hparams[k].append(v)

df = pd.DataFrame(zip(names, method_acc, method_flops, method_comm), columns=["Name", "Method Accuracy", "Method Flops", "Method Comm Cost"])
for k, v in hparams.items():
    df[k] = v + [None] * (len(files) - len(v))
df

,Name,Method Accuracy,Method Flops,Method Comm Cost,cifar,num_clients,k,discount,poll_clients,interrupted,...,epochs,use_ucb,use_random,use_contrastive,num_partitions,use_masked,l1_norm_weight,classwise_subset,num_groups,experiment_name
0,ucb_poll_cl10,"[0.176, 0.20093, 0.19082000000000002, 0.40179,...","[3571366400000, 7142732800000, 10714099200000,...","[2334.725024000035, 4669.450047999885, 7004.17...",True,10,6,0.7,True,False,...,150,True,False,True,1,False,5.000000e-07,False,5,ucb_poll_cl
1,classwise_masked_ucb_poll_interrupted_cl10,"[0.09983333333333334, 0.09983333333333334, 0.0...","[1403225856000, 2806451712000, 4209677568000, ...","[0.013519999999999657, 0.027040000000000994, 0...",True,10,6,0.7,True,True,...,150,True,False,True,1,True,5.000000e-07,True,5,classwise_masked_ucb_poll_interrupted_cl
2,vanilla_classwise_masked10,"[0.7603333333333333, 0.7181666666666667, 0.749...","[3857075712000, 7714151424000, 11571227136000,...","[4202.495999999827, 8404.992000000964, 12607.4...",True,10,6,0.7,False,False,...,150,False,False,False,1,True,5.000000e-07,True,5,vanilla_classwise_masked
3,ucb_poll_interrupted_cl10,"[0.11435, 0.11431, 0.11410000000000001, 0.1142...","[1299283200000, 2598566400000, 3897849600000, ...","[0.012559999999999738, 0.02512000000000074, 0....",True,10,6,0.7,True,True,...,150,True,False,True,1,False,5.000000e-07,False,5,ucb_poll_interrupted_cl
4,vanilla_classwise10,"[0.20466666666666664, 0.277, 0.210166666666666...","[3857075712000, 7714151424000, 11571227136000,...","[4202.495999999827, 8404.992000000964, 12607.4...",True,10,6,0.7,False,False,...,150,False,False,False,1,False,5.000000e-07,True,5,vanilla_classwise
5,random_interrupted_cl10,"[0.11444999999999998, 0.11442999999999999, 0.1...","[1299283200000, 2598566400000, 3897849600000, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True,10,6,0.7,False,True,...,150,False,True,True,1,False,5.000000e-07,False,5,random_interrupted_cl
6,vanilla10,"[0.53208, 0.62365, 0.71129, 0.72793, 0.7371700...","[3571366400000, 7142732800000, 10714099200000,...","[3891.199999999955, 7782.4000000007245, 11673....",True,10,6,0.7,False,False,...,150,False,False,False,1,False,5.000000e-07,False,5,vanilla
7,random_cl10,"[0.19180999999999998, 0.22220999999999996, 0.3...","[3571366400000, 7142732800000, 10714099200000,...","[2334.7200000000457, 4669.439999999909, 7004.1...",True,10,6,0.7,False,False,...,150,False,True,True,1,False,5.000000e-07,False,5,random_cl
8,interrupted_cl10,"[0.11429, 0.11426999999999998, 0.1146300000000...","[1299283200000, 2598566400000, 3897849600000, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True,10,6,0.7,False,True,...,150,False,False,True,1,False,5.000000e-07,False,5,interrupted_cl
9,classwise_masked_ucb_poll10,"[0.6745, 0.685, 0.714, 0.7241666666666666, 0.7...","[3857075712000, 7714151424000, 11571227136000,...","[2521.503007999981, 5043.006016000031, 7564.50...",True,10,6,0.7,True,False,...,150,True,False,False,1,True,5.000000e-07,True,5,classwise_masked_ucb_poll


In [7]:
fig = go.Figure()

for i in range(len(df)):
    fig.add_trace(
        go.Scatter(x=eval(steps[i]), y=eval(df['Method Accuracy'][i]), name=df["Name"][i])
    )
fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

fig.update_layout(template=None, xaxis_title="Training Steps", yaxis_title="Per-Client Test Accuracy", font=dict(family='Times New Roman', size=14, color='Black'))

fig.show()

In [8]:
df_new = pd.DataFrame()
df_new['Name'] = df['Name']
df_new['Accuracy'] = df['Method Accuracy'].apply(lambda x: max(eval(x)))
df_new['GFLOPS'] = df['Method Flops'].apply(lambda x: max(eval(x)) * 1e-9)
df_new['Comm Cost (GBs)'] = df['Method Comm Cost'].apply(lambda x: max(eval(x)) * 1e-3)
df_new.sort_values('Accuracy', ascending=False, inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new

,Name,Accuracy,GFLOPS,Comm Cost (GBs)
0,vanilla_classwise_masked10,0.915167,578561.356800,630.374400
1,interrupted_cl10,0.883730,281231.641600,147.865600
2,vanilla10,0.882800,535704.960000,583.680000
3,classwise_masked_ucb_poll_interrupted_cl10,0.882667,266431.655117,95.818629
4,random_interrupted_cl10,0.881300,246695.976960,88.719360
5,ucb_poll_interrupted_cl10,0.872160,246695.976960,88.720958
6,classwise_masked_ucb_poll10,0.845000,578561.356800,378.225451
7,ucb_poll_cl10,0.616550,535704.960000,350.208754
8,random_cl10,0.606140,535704.960000,350.208000
9,vanilla_classwise10,0.580833,578561.356800,630.374400
